<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chapter-2---Elements-of-Matrix-Theory" data-toc-modified-id="Chapter-2---Elements-of-Matrix-Theory-1">Chapter 2 - Elements of Matrix Theory</a></span><ul class="toc-item"><li><span><a href="#2.1.2-The-Jordan-Normal-Form" data-toc-modified-id="2.1.2-The-Jordan-Normal-Form-1.1">2.1.2 The Jordan Normal Form</a></span><ul class="toc-item"><li><span><a href="#Example-2.5-Revisiting-the-wireless-sensor-network-example" data-toc-modified-id="Example-2.5-Revisiting-the-wireless-sensor-network-example-1.1.1">Example 2.5 Revisiting the wireless sensor network example</a></span></li><li><span><a href="#NumPy/-SciPy-approach" data-toc-modified-id="NumPy/-SciPy-approach-1.1.2">NumPy/ SciPy approach</a></span></li><li><span><a href="#SymPy-approach" data-toc-modified-id="SymPy-approach-1.1.3">SymPy approach</a></span></li></ul></li><li><span><a href="#2.1.3-Semi-convergence-and-convergence-for-discrete-time-linear-systems" data-toc-modified-id="2.1.3-Semi-convergence-and-convergence-for-discrete-time-linear-systems-1.2">2.1.3 Semi-convergence and convergence for discrete-time linear systems</a></span><ul class="toc-item"><li><span><a href="#Definition-2.6-(Spectrum-and-spectral-radius-of-a-matrix)" data-toc-modified-id="Definition-2.6-(Spectrum-and-spectral-radius-of-a-matrix)-1.2.1">Definition 2.6 (Spectrum and spectral radius of a matrix)</a></span></li></ul></li><li><span><a href="#2.2.1-The-spectral-radius-for-row-stochastic-matrices" data-toc-modified-id="2.2.1-The-spectral-radius-for-row-stochastic-matrices-1.3">2.2.1 The spectral radius for row-stochastic matrices</a></span><ul class="toc-item"><li><span><a href="#Theorem-2.8-(Geršgorin-Disks-Theorem)" data-toc-modified-id="Theorem-2.8-(Geršgorin-Disks-Theorem)-1.3.1">Theorem 2.8 (Geršgorin Disks Theorem)</a></span></li></ul></li><li><span><a href="#2.3.3-Applications-to-matrix-powers-and-averaging-systems" data-toc-modified-id="2.3.3-Applications-to-matrix-powers-and-averaging-systems-1.4">2.3.3 Applications to matrix powers and averaging systems</a></span><ul class="toc-item"><li><span><a href="#Theorem-2.13-(Powers-of-non-negative-matrices-with-a-simple-and-strictly-dominant-eigenvalue)" data-toc-modified-id="Theorem-2.13-(Powers-of-non-negative-matrices-with-a-simple-and-strictly-dominant-eigenvalue)-1.4.1">Theorem 2.13 (Powers of non-negative matrices with a simple and strictly dominant eigenvalue)</a></span><ul class="toc-item"><li><span><a href="#Example-2.14-Wireless-sensor-network" data-toc-modified-id="Example-2.14-Wireless-sensor-network-1.4.1.1">Example 2.14 Wireless sensor network</a></span></li></ul></li></ul></li><li><span><a href="#Exercises-2.17-and-2.18" data-toc-modified-id="Exercises-2.17-and-2.18-1.5">Exercises 2.17 and 2.18</a></span></li></ul></li></ul></div>

In [1]:
%matplotlib widget

# Import packages
import scipy.linalg as spla
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sympy import Matrix
# For interactive graphs
import ipywidgets as widgets

# Settings
custom_figsize= (6, 4) # Might need to change this value to fit the figures to your screen
custom_figsize_square = (5, 5)  

In [52]:
def matprint(mat, fmt="g"):
    """
    Own defined function to beautiful print matrices in the output.
    """
    # Handling 1d-arrays
    if np.ndim(mat) == 1:
        mat = mat.reshape(mat.shape[0], 1)
    # Handling column spaces
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]

    for x in mat:
        for i, y in enumerate(x):
            if i == 0:
                print(("|  {:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
            else:                    
                print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("|")


def plot_spectrum(M, ax):
    """
    Scatter plot of eigs in complex plane overlayed by radius circle
    """
    eigvals = spla.eig(M)[0]  # Extract only eigenvalues
    
    ax.scatter(eigvals.real, eigvals.imag)
    ax.set_xlim(-1.05,1.05)
    ax.set_ylim(-1.05,1.05)
    
    r = max(abs(eigvals))  # Note: abs() automatically calculated the magnitude of complex values
    circle=mpl.patches.Circle((0,0),radius=r,alpha=.2,ec='black')
    ax.add_patch(circle)
    ax.axhline(0, color='black', ls='--', linewidth=1)
    ax.axvline(0, color='black', ls='--', linewidth=1)
    return eigvals, ax


def plot_gersgorin_disks(M, ax):
    """
    Scatter plot of eigenvalues overlayed with gersgorin disks (marked with green edges)
    """
    eigvals, ax = plot_spectrum(M, ax)
    
    row_sums = M.sum(axis=1)
    for i in range(M.shape[0]):
        ax.add_patch(mpl.patches.Circle((M[i,i],0),radius=row_sums[i]-M[i,i],
                                    alpha=.6/M.shape[0], ec='green'))

**TODO:**

Chapter 2:
- randomly generate a matrix, compute Jordan form, plot spectrum  -- DONE
- randomly generate a matrix and plot the Gersgorin disks -- DONE
- compute powers of a primitive row-stochastic matrix and show it converges to rank 1 -- DONE
- illustrate dynamics of algorithms 2.17 and 2.18

# Chapter 2 - Elements of Matrix Theory
These Jupyter Notebook scripts contain some examples, visualization and supplements accompanying the book "Lectures on Network Systems" by Francesco Bullo http://motion.me.ucsb.edu/book-lns/. These scripts are published with the MIT license. **Make sure to run the first cell above to import all necessary packages and functions and adapt settings in case.** In this script it is necessary to execute cell by cell chronologically due to reocurring examples. (Tip: Use the shortcut Shift+Enter to execute each cell). Most of the functions are kept in separate files to keep this script neat.

## 2.1.2 The Jordan Normal Form
### Example 2.5 Revisiting the wireless sensor network example
The following cells are showing the computation of the Jordan Normal Form $J$, the invertible transformation matrix $T$ and some of its dependencies. 

In [3]:
# Defining the A matrix again
A = np.array([[1/2, 1/2, 0., 0.],
              [1/4, 1/4, 1/4, 1/4],
              [0., 1/3, 1/3, 1/3],
              [0., 1/3, 1/3, 1/3]
])

There is the possibility to calculate the Jordan Normal Form directly with the package SymPy https://docs.sympy.org/latest/index.html. However, we are determining the Jordan Normal Form via determining the generalized eigenvectors (read more for literature recommendations about generalized eigenvectors in the book) with the SciPy package first to discuss some possibilities and problems with non symbolic toolboxes.

### NumPy/ SciPy approach

From the documentation of scipy.linalg.eig: *'Solve an ordinary or generalized eigenvalue problem of a square matrix.'*

In [4]:
# Right eigenvectors
lambdas, eigv = spla.eig(A)

# Left eigenvectors
lambdas2, eigw = spla.eig(A.T)

Due to numerical instabilities, the zero values are not reflected and it can be seen, how the expected eigenvalue of 1 is not precise. The zeros can be fixed with:

In [5]:
def correct_close_to_zero(M, tol=1e-12):
    M.real[abs(M.real) < tol] = 0.0
    if M.imag.any():
        M.imag[abs(M.imag) < tol] = 0.0
    return M

eigv_cor = correct_close_to_zero(eigv)
eigw_cor = correct_close_to_zero(eigw)
lambdas_cor = correct_close_to_zero(lambdas)
lambdas2_cor = correct_close_to_zero(lambdas2)

print("Right eigenvectors:")
matprint(eigv_cor)
print("\n")
print("Left eigenvectors:")
matprint(eigw_cor)
print("\n")
print("Eigenvalues (right):")
matprint(lambdas_cor)
print("\n")
print("Eigenvalues (left) for matching later:")
matprint(lambdas2_cor)

Right eigenvectors:
|  -0.5  0.855025   0.555542          0  |
|  -0.5  0.110013  -0.719612          0  |
|  -0.5  -0.35835   0.294561  -0.707107  |
|  -0.5  -0.35835   0.294561   0.707107  |


Left eigenvectors:
|  0.324443  -0.733894  -0.333766          0  |
|  0.648886  -0.188856   0.864677          0  |
|  0.486664   0.461375  -0.265456  -0.707107  |
|  0.486664   0.461375  -0.265456   0.707107  |


Eigenvalues (right):
|          1+0j  |
|   0.564333+0j  |
|  -0.147667+0j  |
|          0+0j  |


Eigenvalues (left) for matching later:
|          1+0j  |
|   0.564333+0j  |
|  -0.147667+0j  |
|          0+0j  |


There are two options now for $T^{-1}$: Taking the inverse of the right eigenvectors (which contains again numerical instabilities) or building it from the left eigenvectors, what would include some sorting to match the eigenvalue order from the  right eigenvector (often it is the case, that they are already aligned since calling scipy.linalg.eig twice on a matrix with the same eigenvalues).

In [6]:
T = eigv_cor.copy()*-1  # Rescale the eigenvectors to match eigenvalues later
# Sorting if necessary, remember to use transpose, since in T^-1 the rows represent the left eigenvectors.
Tinv = eigw_cor.T.copy()

Now we can simply compute J, when compared, is fairly close to the solution in the book, however, due to numerical intabilities not precise. Further on, the order of the eigenvalues might be different than the on from the book.

In [7]:
J = correct_close_to_zero(Tinv@A@T)
print("Jordan Normal Form via SciPy/Numpy:")
matprint(J)

Jordan Normal Form via SciPy/Numpy:
|  0.973329        0          0  0  |
|         0  0.55245          0  0  |
|         0        0  -0.142357  0  |
|         0        0          0  0  |


### SymPy approach

Now we use a symbolic toolbox package SymPy from python as a blackbox. Note, that also here the order of the eigenvalues might be different!

In [8]:
Asym = Matrix(A) # Sympy Matrix toolbox object
Tsym, Jsym = Asym.jordan_form()

Here we can compare them with our previous results:

In [9]:
print("Jordan Normal Form SymPy:")
matprint(np.array(Jsym).astype(np.float64))
print("Jordan Normal Form SciPy:")
matprint(J)

Jordan Normal Form SymPy:
|  0  0          0         0  |
|  0  1          0         0  |
|  0  0  -0.147667         0  |
|  0  0          0  0.564333  |
Jordan Normal Form SciPy:
|  0.973329        0          0  0  |
|         0  0.55245          0  0  |
|         0        0  -0.142357  0  |
|         0        0          0  0  |


## 2.1.3 Semi-convergence and convergence for discrete-time linear systems


### Definition 2.6 (Spectrum and spectral radius of a matrix)
We display the spectrum of the previous A matrix with the spectrum radius for visualization purpose. Additionally, we also show how the spectrum of a randomly generated matrix.

In [53]:
fig, ax213 = plt.subplots(figsize=custom_figsize_square)
plot_spectrum(A, ax213);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [54]:
n_M1=8

# A unifornmly distributed, positive, row stochastic matrix vs not row stochastic
M1 = np.random.uniform(0, 1,(n_M1,n_M1))
M1 = M1 / M1.sum(axis=1, keepdims=1) # Row-stochastic
M2 = M1 - 0.05 # Not row-stochastic

fig, (ax2131, ax2132) = plt.subplots(1,2, figsize=(custom_figsize_square[0]*2, custom_figsize_square[1]))
plot_spectrum(M1, ax2131);
plot_spectrum(M2, ax2132);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 2.2.1 The spectral radius for row-stochastic matrices
### Theorem 2.8 (Geršgorin Disks Theorem)

Similar to before, the Geršgorin Disks are now visualized for a row-stochastic matrix and another matrix.

In [55]:
fig, (ax2211, ax2212) = plt.subplots(1,2, figsize=(custom_figsize_square[0]*2, custom_figsize_square[1]))
plot_gersgorin_disks(M1, ax2211)
plot_gersgorin_disks(M2, ax2212)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 2.3.3 Applications to matrix powers and averaging systems

### Theorem 2.13 (Powers of non-negative matrices with a simple and strictly dominant eigenvalue)

Here is an example for Theorem 2.13, which shows, how the powers of primitive, row-stochastic matrices converges to rank 1. This is also done for the wireless sensor network example.

#### Example 2.14 Wireless sensor network
In the book it is shown, that the wireless sensor network matrix is primitive. Here, the eigenvectors and eigenvalues are printed again and compared for the semi convergence result $\lim_{k \to \infty} A^k = \mathbb{1}_n w^T$ to demonstrate Theorem 2.13 for a row-stochastic matrix. 

In [112]:
print("Left eigenvectors of A:")
matprint(eigw_cor)
print("\n")

print("Eigenvalues (left) of A:")
matprint(lambdas2_cor)
print("\n")

print("Normalizing dominant eigenvector:")
dom_eigv = eigw_cor[:, 0] / sum(eigw_cor[:, 0])
matprint(dom_eigv)
print("\n")

print("Convergence result of A:")
matprint(np.linalg.matrix_power(A, 50))
print("\n")

print("equals 1n*w^T")
matprint(np.ones((4,1))@dom_eigv[:, None].T)

Left eigenvectors of A:
|  0.324443  -0.733894  -0.333766          0  |
|  0.648886  -0.188856   0.864677          0  |
|  0.486664   0.461375  -0.265456  -0.707107  |
|  0.486664   0.461375  -0.265456   0.707107  |


Eigenvalues (left) of A:
|          1+0j  |
|   0.564333+0j  |
|  -0.147667+0j  |
|          0+0j  |


Normalizing dominant eigenvector:
|  0.166667  |
|  0.333333  |
|      0.25  |
|      0.25  |


Convergence result of A:
|  0.166667  0.333333  0.25  0.25  |
|  0.166667  0.333333  0.25  0.25  |
|  0.166667  0.333333  0.25  0.25  |
|  0.166667  0.333333  0.25  0.25  |


equals 1n*w^T
|  0.166667  0.333333  0.25  0.25  |
|  0.166667  0.333333  0.25  0.25  |
|  0.166667  0.333333  0.25  0.25  |
|  0.166667  0.333333  0.25  0.25  |


Below is a randomly generated example to show, that primitive, row-stochastic matrices always converge to rank 1. *Note: The code is not robust for semisimple eigenvalue of 1*

In [140]:
# Creating a new random primitive (positiv), row stochastic matrix here
n_M11=5
M11 = np.random.uniform(0, 1,(n_M11,n_M11))
M11 = M11 / M11.sum(axis=1, keepdims=1) # Row-stochastic
print("Random matrix M:")
matprint(M11)
print("\n")

print("Left eigenvectors of M:")
l_M, m_eigv = spla.eig(M11.T)
m_eigv = correct_close_to_zero(m_eigv)
l_M = correct_close_to_zero(l_M)
matprint(m_eigv)
print("\n")

print("Eigenvalues (left) of M:")
matprint(l_M)
print("\n")

# Here we check the position with numerical unprecision of the eigenvalue 1
print("Normalizing dominant eigenvector:")
idx_dom = np.where(abs(l_M - 1) < 0.005)[0][0]
dom_eigv_M = m_eigv[:, 0] / sum(m_eigv[:, 0])
matprint(dom_eigv_M)
print("\n")

print("Convergence result of M:")
matprint(np.linalg.matrix_power(M11, 500))
print("\n")

print("equals 1n*w^T")
matprint(np.ones((n_M11,1))@dom_eigv_M[:, None].T)

Random matrix M:
|  0.0909095  0.168988  0.210201   0.225077   0.304824  |
|   0.553929  0.157486  0.112857   0.140981  0.0347468  |
|   0.150733  0.301491  0.183454     0.2867  0.0776224  |
|  0.0973755  0.319085  0.187092   0.274636   0.121812  |
|   0.206725  0.369428  0.259951  0.0766552  0.0872401  |


Left eigenvectors of M:
|  0.509948+0j         -0.664106+0j         -0.664106+0j    0.165715+0.0484966j    0.165715-0.0484966j  |
|  0.541953+0j    0.104622-0.44723j    0.104622+0.44723j  -0.0990081+0.0127006j  -0.0990081-0.0127006j  |
|  0.400592+0j   0.177359-0.112735j   0.177359+0.112735j     0.484873-0.148978j     0.484873+0.148978j  |
|  0.452055+0j  0.0310776+0.173079j  0.0310776-0.173079j           -0.796895+0j           -0.796895+0j  |
|  0.285328+0j   0.351047+0.386887j   0.351047-0.386887j    0.245316+0.0877811j    0.245316-0.0877811j  |


Eigenvalues (left) of M:
|                 1+0j  |
|  -0.150443+0.252812j  |
|  -0.150443-0.252812j  |
|  0.047306+0.0292098j  |
|  0.0

## Exercises 2.17 and 2.18

In [106]:
np.ones((4,1))

array([[1.],
       [1.],
       [1.],
       [1.]])